In [2]:
import pandas as pd
import requests
from io import StringIO

In [3]:
#Definir función de extracción con scrapíng puesto que entraremos a varios links
#Adicional la API de datos.gov.co (Socrata) tiene un límite de 1000 filas, por lo cuál se debe ingresar por 
#peticiones a través de filtros

def exported(base_url, treat_serial_as_str=False):
    #Acceder al link desado
    response = requests.get(base_url)
    # Obtener el año actual
    current_year = pd.to_datetime("today").year

    # Crear una lista para almacenar los DataFrames por año
    dataframes = []

    # Iterar a través de los años desde 2019 hasta el año actual
    for year in range(2019, current_year + 1):
        # Construir la URL para el año actual
        year_url = f"{base_url}?$where=fecha between '{year}-01-01T00:00:00' and '{year}-12-31T00:00:00'&$limit=10000000"
        # Realizar una solicitud GET a la URL y obtener el contenido del CSV
        response = requests.get(year_url)

        # Crear un DataFrame a partir del contenido del CSV
        df = pd.read_csv(StringIO(response.text))

        # Agregar el DataFrame a la lista
        dataframes.append(df)

    # Concatenar todos los DataFrames en uno solo
    data = pd.concat(dataframes, ignore_index=True)
    data.columns = map(str.upper, data.columns)
    data["FECHA"] = pd.to_datetime(data["FECHA"])

    return data

#Como hay un documento con el campo de fecha nombrado de forma diferente, es necesario "duplicar" la función para poder
#interactuar con este

def exported2(base_url):
    #Acceder al link desado
    response = requests.get(base_url)
    # Obtener el año actual
    current_year = pd.to_datetime("today").year

    # Crear una lista para almacenar los DataFrames por año
    dataframes = []

    # Iterar a través de los años desde 2019 hasta el año actual
    for year in range(2019, current_year + 1):
        # Construir la URL para el año actual
        year_url = f"{base_url}?$where=fecha_comp between '{year}-01-01T00:00:00' and '{year}-12-31T00:00:00'&$limit=10000000"
        # Realizar una solicitud GET a la URL y obtener el contenido del CSV
        response = requests.get(year_url)

        # Crear un DataFrame a partir del contenido del CSV
        df = pd.read_csv(StringIO(response.text))

        # Agregar el DataFrame a la lista
        dataframes.append(df)

    # Concatenar todos los DataFrames en uno solo
    data = pd.concat(dataframes, ignore_index=True)
    data.columns = map(str.upper, data.columns)
    data["FECHA_COMP"] = pd.to_datetime(data["FECHA_COMP"])

    return data


In [4]:
#Documento 1, con información disponible desde el 2022 y casi llegando a fecha actual del 2024
url = "https://www.datos.gov.co/resource/e7nm-5ibv.csv"
#Implementar función creada para extraer la información y crear dataframe
comparendos1 = exported(url)

#La primera extracción transforma carácteres especiales en _, por lo que se renombran para mayor facilidad de uso y distinción
comparendos1 = comparendos1.rename(columns={"INFRACCI_N": "INFRACCION",
                                            "LUGAR_INFRACCI_N": "LUGAR INFRACCION"})

#Con la intención de juntar los 3 documentos en un solo dataframe, en éste se crea un index para identificar 
#los comparendos de forma única, basados en la fecha en que se impusieron
comparendos1["fcid"] = comparendos1["FECHA"].astype(str).str.replace('-', '').astype(int)

#comparendos1.info()
# Inicializar un contador para agregar caracteres numéricos únicos
contador = 1

# Función para generar el índice único basado en la fecha y el contador
def generar_id(fecha):
    global contador
    id_unico = str(fecha) + str(contador).zfill(4)
    contador += 1
    return id_unico

# Aplicar la función a la columna 'fcid creado para la nueva columna 'COMPARENDO'
comparendos1['COMPARENDO'] = comparendos1['fcid'].apply(generar_id)

#Agregar una columna en nulos, que puede interesar dado que estás disponible en los otros 2 archivos
comparendos1["PLACA"] = None

#Poner en mayúsculas los valores de las siguientes columnas, para unificar formatos con los demás documentos
comparendos1["SERVICIO"] = comparendos1["SERVICIO"].str.upper()
comparendos1["ESTADO"] = comparendos1["ESTADO"].str.upper()

#Seleccionar columnas deseadas y de interés
comparendos1 = comparendos1[["COMPARENDO", "FECHA", "CLASE", "SERVICIO", "ESTADO", "INFRACCION", "PLACA"]]

comparendos1.head()

,COMPARENDO,FECHA,CLASE,SERVICIO,ESTADO,INFRACCION,PLACA
0,202201010001,2022-01-01,MOTOCICLETA,PARTICULAR,RESOLUCION,C35,None
1,202201010002,2022-01-01,MOTOCICLETA,PARTICULAR,PAGADO,C35,None
2,202201020003,2022-01-02,AUTOMOVIL,PARTICULAR,RESOLUCION,C02,None
3,202201020004,2022-01-02,MOTOCICLETA,PARTICULAR,RESOLUCION,C15,None
4,202201020005,2022-01-02,MOTOCICLETA,PARTICULAR,RESOLUCION,D01,None


In [5]:
#Documento 2, con información disponible desde el 2020
url2 = "https://www.datos.gov.co/resource/rfag-apa4.csv"
#Implementar función creada para extraer la información y crear dataframe
comparendos2 = exported(url2)

#Agregar columnas requeridas y llenar con valores nulos para incluirlas dentro del documento final
comparendos2["CLASE"] = None
comparendos2["ESTADO"] = None
comparendos2["SERVICIO"] = None

#Seleccionar columnas desdeadas
comparendos2 = comparendos2[["COMPARENDO", "FECHA", "CLASE", "SERVICIO", "ESTADO", "INFRACCION", "PLACA"]]

comparendos2.head()

,COMPARENDO,FECHA,CLASE,SERVICIO,ESTADO,INFRACCION,PLACA
0,20088524,2020-02-01,None,None,None,C35,BSC21D
1,20088718,2020-02-01,None,None,None,C02,NaN
2,20088719,2020-02-01,None,None,None,C24,NaN
3,20095360,2020-02-01,None,None,None,D02,WFC018
4,20088888,2020-03-01,None,None,None,C02,GLE19A


In [6]:
#Documento 3, con información del 2021
url3 = "https://www.datos.gov.co/resource/wmr7-xdpj.csv"
#Implementar función creada para extraer la información y crear dataframe
comparendos3 = exported2(url3)

#Renombrar columnas deseadas para normalizarlas acorde a los demás archivos
comparendos3 = comparendos3.rename(columns= {"FECHA_COMP": "FECHA",
                                             "NRO_COMPARENDO": "COMPARENDO"})
#Seleccionar columnas deseadas
comparendos3 = comparendos3[["COMPARENDO", "FECHA", "CLASE", "SERVICIO", "ESTADO", "INFRACCION", "PLACA"]]
comparendos3.head()

,COMPARENDO,FECHA,CLASE,SERVICIO,ESTADO,INFRACCION,PLACA
0,99999999000004761363,2021-01-01,AUTOMOVIL,PARTICULAR,CANCELADO,C14,BVD924
1,68276000000020095790,2021-01-01,MOTOCICLETA,PARTICULAR,RESOLUCION,D01,CAP56F
2,99999999000004761362,2021-01-01,MOTOCICLETA,PARTICULAR,CANCELADO,C14,TKI80E
3,68276000000020090352,2021-01-02,NaN,NaN,ANULADO,D02,XLP273
4,68276000000020090353,2021-01-02,BUSETA,PUBLICO,CANCELADO,C35,XLF273


In [7]:
#Concatenar en un sólo documento
full_doc = pd.concat([comparendos2, comparendos3, comparendos1], ignore_index= True)

full_doc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24324 entries, 0 to 24323
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   COMPARENDO  24324 non-null  object        
 1   FECHA       24324 non-null  datetime64[ns]
 2   CLASE       20740 non-null  object        
 3   SERVICIO    20734 non-null  object        
 4   ESTADO      20752 non-null  object        
 5   INFRACCION  24317 non-null  object        
 6   PLACA       8988 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 1.3+ MB


In [8]:
full_doc.head()

,COMPARENDO,FECHA,CLASE,SERVICIO,ESTADO,INFRACCION,PLACA
0,20088524,2020-02-01,None,None,None,C35,BSC21D
1,20088718,2020-02-01,None,None,None,C02,NaN
2,20088719,2020-02-01,None,None,None,C24,NaN
3,20095360,2020-02-01,None,None,None,D02,WFC018
4,20088888,2020-03-01,None,None,None,C02,GLE19A


In [10]:
#Extraer información sobre las infracciones directamente desde el simit de Colombia

from lxml import html
from bs4 import BeautifulSoup as bs
import re


url_comparendos = "https://www.comparendossimit.com/infracciones-de-transito-colombia/"

encabezado = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

#Este encabezado puede ser genérico sin embargo hay que indagar si presenta modificaciones según caso de uso

datos = requests.get(url_comparendos, headers= encabezado)

datos2 = html.fromstring(datos.text)
soup = bs(datos.text, "html.parser")
cods = []
# Encuentra todos los elementos con la clase "mtr-cell-content"
codigos = soup.find_all(class_="mtr-cell-content")
for codigo in codigos:
    l = codigo.text  # Usa el atributo text, no una función
    cods.append(l)
#print(cods)

tripleta = [cods[i:i+3] for i in range (3, len(cods), 3)]

Costos = pd.DataFrame(tripleta, columns= ['CÓDIGO', 'TIPO DE INFRACCIÓN', 'VALOR'])

#Para casos funcionales, es mejor mantener la primera columna siempre de 3 carácteres
# Función para formatear los valores de la primera columna
def formatear_codigo(valor):
    partes = re.match(r'([A-Za-z]+)(\d+)', valor)
    if partes:
        prefijo = partes.group(1)
        numero = int(partes.group(2))
        if numero < 10:
            numero_formateado = f'0{numero}'
        else:
            numero_formateado = str(numero)
        return f'{prefijo}{numero_formateado}'
    return valor

# Aplicar la función a la primera columna del DataFrame
Costos['CÓDIGO'] = Costos['CÓDIGO'].apply(formatear_codigo)

Costos.head(20)

,CÓDIGO,TIPO DE INFRACCIÓN,VALOR
0,A01,No transitar por la derecha de la vía,$173.733
1,A02,Agarrarse de otro vehículo en movimiento,$173.733
2,A03,Transportar personas o cosas que disminuyen su...,$173.733
3,A04,"Transitar por andenes, aceras, puentes o demás...",$173.733
4,A05,No respetar las señales de tránsito,$173.733
5,A06,Transitar sin los dispositivos luminosos reque...,$173.733
6,A07,Transitar sin dispositivos que permitan la par...,$173.733
7,A08,"Transitar por zonas prohibidas o, por aquellas...",$173.733
8,A09,Adelantar entre dos (2) vehículos automotores ...,$173.733
9,A10,Conducir por la vía férrea o por zonas de prot...,$173.733
